#### Day 18 - B
Find the coordinates of the first byte to make the maze impossible to solve

In [28]:
#Import Libraries and settings
import bisect
from copy import deepcopy

settings = {
    "day": 18,
    "test_data": 0
}

In [29]:
#Load Input
def load_input(settings):
    #Derrive input file name
    if settings["test_data"]:
        data_subdir = "test"
        grid_dimen = (7,7)
        to_fall = 12
    else:
        data_subdir = "actual"
        grid_dimen = (71,71)
        to_fall = 1024

    data_fp = f"./../input/{data_subdir}/{settings["day"]}.txt"

    #Open and read the file
    with open(data_fp) as f:
        lines = f.read().split('\n')

    byte_loc = []
    for line in lines:
        coords = line.split(",")
        byte_loc.append((int(coords[0]), int(coords[1])))


    return byte_loc, grid_dimen, to_fall

byte_loc, grid_dimen, to_fall = load_input(settings)
GRID_DIMENSIONS = grid_dimen
BYTES_TO_FALL = to_fall

In [30]:
#Convert compass direction into a dir tuple
def translate_step(dir_g, mag=1):
    v = 0
    h = 0

    if "^" in dir_g:
        v = -mag
    elif "v" in dir_g:
        v = mag

    if ">" in dir_g:
        h = mag
    elif "<" in dir_g:
        h = -mag

    return (h, v)

#Apply a direction to a location
def apply_move(loc, dir, mag=1):
    step = translate_step(dir, mag)
    return (loc[0]+step[0], loc[1]+step[1])

#Get all neighbours to consider for a space
def get_neighbours(grid, loc, history):
    dirs = ["^", ">", "v", "<"]
    nbs = []
    base_cost = history[loc]

    for pos_dir in dirs:
        next_loc = apply_move(loc, pos_dir)
        if (
            next_loc[0] >= 0 and 
            next_loc[0] < GRID_DIMENSIONS[0] and
            next_loc[1] >= 0 and
            next_loc[1] < GRID_DIMENSIONS[1]):
            if grid[next_loc[1]][next_loc[0]] != "#":
                if next_loc not in history.keys():
                    next_cost = base_cost + 1
                    history[next_loc] = next_cost
                    nbs.append((next_loc, next_cost))

    return nbs


In [31]:
len(byte_loc)

3450

In [32]:
#Build a grid with fallen bytes
def build_grid(bytez, grid_dimen):
    #Build empty grid
    grid = []
    for idx_y in range(grid_dimen[1]):
        row = []
        for idx_x in range(grid_dimen[0]):
            row.append(".")
        grid.append(row)

    #Populate start and end spaces
    grid[0][0] = "S"
    grid[grid_dimen[1]-1][grid_dimen[0]-1] = "E"

    for byte in bytez:
        grid[byte[1]][byte[0]] = "#"

    return grid

In [33]:
#Algorithm to find the minimum path between S and E
def breadth_first(bytez, starting_loc=(0,0), end_loc=(GRID_DIMENSIONS[0]-1, GRID_DIMENSIONS[1]-1)):
    grid = build_grid(bytez, GRID_DIMENSIONS)

    #Track the history of which spaces were visited
    history = {starting_loc:0}
    #Maintain a queue of spaces to explore
    queue = [(starting_loc, 0)]
    
    #Iterate while there are spaces to explore
    while queue:
        next_move = queue[0]
        loc = next_move[0]
        queue.pop(0)

        #Get all neighbouring spaces that are unexplored and open to move to
        nbs = get_neighbours(grid, loc, history)
        for future_move in nbs:
            #If the end of the maze is found, return the number of steps
            if future_move[0] == end_loc: 
                return grid, future_move[1]
            #Add next spaces to explore to the queue ordered by cost (number of steps)
            bisect.insort(queue, future_move, key=lambda x:x[1])

    return False

In [42]:
#Iterate until no solution found
offset = 1900 #Set to 1900 as it was found there is a solution for the first 1900 bytes
res = True
while res:
    offset += 1
    if offset % 100 == 0:
        print(offset)
    res = breadth_first(byte_loc[:BYTES_TO_FALL + offset])
    if res:
        grid_fin = res[0]

#Print output
print(BYTES_TO_FALL + offset, "bytes fell. Delta from 1024:", offset)
print(byte_loc[BYTES_TO_FALL + offset - 1])

2954 bytes fell. Delta from 1024: 1930
(46, 28)
